# get image charaters 

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
from copy import deepcopy
import string
from datetime import datetime 

In [0]:
def get_coords(gray_image, dilation_iters=2):
  #gray_image = cv2.ximgproc.l0Smooth(gray_image, 0.02, 2)
  ret, thresh1 = cv2.threshold(gray_image,190,255,cv2.THRESH_BINARY_INV)
  kernel = np.ones((5,5),np.uint8)
  thresh1 = cv2.dilate(thresh1, kernel, iterations=dilation_iters)
  _, contours, hierarchy = cv2.findContours(thresh1,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
  #if dilation_iters>0: plot_image(thresh1, 'gray')
  cordinates = []
  for cnt in contours:
      x,y,w,h = cv2.boundingRect(cnt)     # (x,y) - left down corner (plt), upper(cv2)
      cordinates.append((x,y,w,h))
  return np.asarray(cordinates)


def plot_bboxes(cordinates, image, write=False, file_path=None):
  for (x,y,w,h) in cordinates:
      #bound the images
      cv2.rectangle(image, (x,y), (x+w,y+h), (0,255,0), 2)
  if write:
    cv2.imwrite(file_path, image)

    
def plot_image(image, colorspace=0):
  if colorspace:
    plt.imshow(image, colorspace)
  else:
    plt.imshow(image) 
  plt.show()

    
# всі літери мають однаковий рівень висоти внизу але різний вверху
# можна зберегти різницю висот слова і літери h_diff
# змінити y-=hdiff, h+=hdiff                                                           !!!
def get_letters(word_coords, image):
  all_letters = []
  for (x,y,w,h) in word_coords:
    word_image = image[y:y+h, x:x+w]
    letter_coords = get_coords(word_image, dilation_iters=0)
    if len(letter_coords) > 0:
      letter_coords = letter_coords[np.logical_and(letter_coords[:,2]>0, letter_coords[:,3]>30)] # w>13px and h>13px
      
      letter_coords[:,0]+=x
      letter_coords[:,1]+=y
      h_difference = abs(h-letter_coords[:,3])
      letter_coords[:,1]-=h_difference # y-h_diff
      letter_coords[:,3]+=h_difference # h+h_diff
      all_letters+= list(letter_coords)
  return np.asarray(all_letters) 


# all_letters - масив координат bbox літер [(x,y,w,h), ]
# big_letters - літери w котрих відхиляються від інших більше ніж на 2.5 * std
# big_indexes - індекси big_letters ві all_letters
def get_big_letters(all_letter_coords):
  big_letters=[]
  big_indexes=[]
  letter_widths = all_letters[:,2]
  width_std = np.std(letter_widths)
  width_mean = np.mean(letter_widths)
  for i in range(len(letter_widths)):
    if letter_widths[i]-width_mean > width_std*2.5:
      big_letters.append(all_letters[i])
      big_indexes.append(i)    
  return big_letters, big_indexes


# dir_path - шлях до директорії із зображеннями jpg
# uniq_char_paths - {назва_зображення: його_шлях}
def get_uniq_char_paths(dir_path):
  uniq_char_paths = {}
  all_char_paths = os.listdir(dir_path)
  for i in range(len(all_char_paths)):
    tokens = all_char_paths[i].split('.')
    if tokens[-1] == 'jpg' or 'JPG':
      char = tokens[-2]
      if len(char)>0:
        char = char[-1]
      else: 
        char = '.'
      uniq_char_paths[char]=os.path.join(dir_path, all_char_paths[i]) 
  return uniq_char_paths


# char_paths_dict - {назва_зображення: його_шлях}
# uniq_chars_dict - {назва_зображення: зображення}
def get_chars_by_paths(char_paths_dict):
  uniq_chars_dict = {}
  for (char, path) in char_paths_dict.items():
    if char in (string.ascii_letters + string.digits):
      uniq_chars_dict[char] = cv2.imread(path, 0)
  return uniq_chars_dict
  

# images - {назва_зображення: зображення} або [зображення, ]
def save_detected_letters(images, save_dir_path, images_dict=0):
  if images_dict:
    for name,img in images.items():
        full_path = os.path.join(save_dir_path, (name+'.jpg'))
        cv2.imwrite(full_path, img)                                          # !!! change trash 
  else:
    for img in images:
        img_name = str(datetime.now()) + '.jpg'
        full_path = os.path.join(save_dir_path, img_name)
        cv2.imwrite(full_path, img)


def get_images_by_coords(coords, image):
  images = []
  for (x,y,w,h) in coords:
    images.append(image[y:y+h, x:x+w])
  return images


# images_dict - {назва_зображення: зображення}
# final_size - (w, h) кінцевого зображення (final_size > image.shape)
# padded_images_dict - {назва_зображення: зображення} (заповнене білим шумом до final_size)
def pad_images(images_dict, final_size):
  h, w = final_size
  padded_images_dict={}
  for name,img in images_dict.items():
    h1, w1 = img.shape
    w_diff, h_diff = abs(w1-w), abs(h1-h)
    w_left = int(w_diff/2)
    w_right = w_diff-w_left
    h_up = int(h_diff/2)
    h_down = h_diff-h_up
    image = np.pad(img, [(h_up, h_down), (w_left, w_right)], 'constant', constant_values= (255,)) # (верх низ) (ліво право)
    padded_images_dict[name] = image
  return padded_images_dict
  
  
# images_dict - {назва_зображення: зображення} 
# ratio - множник ширини та висоти зображення
# resized_images_dict - {назва_зображення: ресайзнуте_зображення}
def resize_images(images_dict, ratio=1):
  if ratio<1:
    interpolation = cv2.INTER_AREA # краще для зменшення
  else:
    interpolation = cv2.INTER_CUBIC # краще для збільшення
  resized_images_dict = {}
  for name,img in images_dict.items():
    h, w = img.shape
    image = cv2.resize(img, (int(w*ratio), int(h*ratio)), interpolation=interpolation)
    resized_images_dict[name]=image
  return resized_images_dict


# dir_path - шлях до директорії зі зображеннями
# train_x, train_y - вектори зображеннь та їх назв відповідно (ndarrays)
def get_images_and_names(dir_path):
  char_paths_dict = get_uniq_char_paths(dir_path)
  char_images_dict = get_chars_by_paths(char_paths_dict)
  images = np.stack(list(char_images_dict.values()), axis=0)
  names = np.stack(list(char_images_dict.keys()), axis=0)
  return images, names



In [0]:
train_path1 =  r'/content/drive/My Drive/Colab Notebooks/course_project/data/train_roboto_mono2'
train_path2 =  r'/content/drive/My Drive/Colab Notebooks/course_project/data/train_helvetica_neue2'

train_x, train_y = get_train_data(train_path2)
print(train_x.shape, train_y.shape)

(52, 32, 32) (52,)


In [0]:
fresh_train_path1 =  r'/content/drive/My Drive/Colab Notebooks/course_project/data/train_roboto_mono'
fresh_train_path2 =  r'/content/drive/My Drive/Colab Notebooks/course_project/data/train_helvetica_neue'

char_paths_dict = get_uniq_char_paths(fresh_train_path2)
char_images_dict = get_chars_by_paths(char_paths_dict)

resized_images_dict = resize_images(char_images_dict, ratio=1/2.5)
padded_images_dict = pad_images(resized_images_dict, (32, 32))

save_letters(padded_images_dict, train_path2, images_dict=1)

In [0]:
# size X 3; dilation=3
image = cv2.imread(r'/content/drive/My Drive/Colab Notebooks/course_project/data/alphabet_helvetica_neue.JPG', 0)


w,h = image.shape[1], image.shape[0]
image = cv2.resize(image, (w*3, h*3), interpolation = cv2.INTER_CUBIC)
word_coords = get_coords(deepcopy(image), dilation_iters=3)
letter_coords = get_letters(word_coords, deepcopy(image))
letter_images = get_images_by_coords(letter_coords, deepcopy(image))


#big_letters, big_indexes = get_big_letters(letter_coords)
#
#char_paths = get_uniq_char_paths(r'/content/drive/My Drive/Colab Notebooks/course_project/train_data')
#chars = get_chars_by_paths(char_paths)
#fixed_letters = fix_strange_letters(big_letters, deepcopy(image), chars)

In [0]:
save_path1 = r'/content/drive/My Drive/Colab Notebooks/course_project/images/WORDS.png'
save_path2 = r'/content/drive/My Drive/Colab Notebooks/course_project/images/WORD_LETTERS.png'
save_path3 = r'/content/drive/My Drive/Colab Notebooks/course_project/images/STRANGE_LETTERS.png'
save_path4 = r'/content/drive/My Drive/Colab Notebooks/course_project/images/ORIGINAL.png'

plot_bboxes(word_coords, deepcopy(image), 1, save_path1)
plot_bboxes(letter_coords, deepcopy(image), 1, save_path2)
plot_bboxes(big_letters, deepcopy(image), 1, save_path3)
cv2.imwrite(save_path4, image)

True

In [0]:

'''
# alternative to get_letters
letter_coords = get_coords(image1, dilation_iters=0)
plot_bboxes(letter_coords, image1)
cv2.imwrite(save_path1, image1)
'''

# -----------------------------------------------------

In [0]:
file_path = r'/content/drive/My Drive/Colab Notebooks/course_project/images/text1.png'
file_path1 = r'/content/drive/My Drive/Colab Notebooks/course_project/test_data/test_x/x_digit/39.JPG'

im1 = cv2.imread(file_path,0)
im = cv2.imread(file_path)
ret,thresh1 = cv2.threshold(im1,180,278,cv2.THRESH_BINARY)
_,contours, hierarchy = cv2.findContours(thresh1,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
# words bounding
for cnt in contours:
	x,y,w,h = cv2.boundingRect(cnt)
	#bound the images
	cv2.rectangle(im,(x,y),(x+w,y+h),(0,255,0),1)

plot_image(im)
#cv2.imwrite(r'/content/drive/My Drive/Colab Notebooks/course_project/images/NEW.jpg',im)

'''
i=0
for cnt in contours:
	x,y,w,h = cv2.boundingRect(cnt)
	#following if statement is to ignore the noises and save the images which are of normal size(character)
	#In order to write more general code, than specifying the dimensions as 100,
	# number of characters should be divided by word dimension
	if w>100 and h>100:
		#save individual images
		cv2.imwrite(str(i)+".jpg",thresh1[y:y+h,x:x+w])
		i=i+1
'''


**погано розпізнається текст, якщо шрифт зєднує букви. У быльшості випадків з'єднання тонкі порівняно з основним текстом, то можна зробити ділейт але оскільки слова малі він їх знищує. треба спочатку ресайзнути слова а тоді ділейт**